## Start


In [16]:
# transform data into sqlite database
from data_loader import TableLoader

table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)



In [17]:
len(table_loader.dataset)

1998

In [1]:
from datasets import load_dataset
totto = load_dataset('./data_loader/totto_zh.py', verification_mode="no_checks", cache_dir="/media/disk2/datasets")
tabfact = load_dataset('./data_loader/tabfact.py', verification_mode="no_checks", cache_dir="/media/disk2/datasets")
sqa = load_dataset('./data_loader/sqa.py', verification_mode="no_checks", cache_dir="/media/disk2/datasets")

In [34]:
filtered_dataset = table_loader.dataset.filter(lambda example: example['small_test'])

Filter: 100%|██████████| 1000/1000 [00:00<00:00, 4669.79 examples/s]


In [35]:
filtered_dataset

Dataset({
    features: ['id', 'table', 'statement', 'label', 'hardness', 'small_test'],
    num_rows: 147
})

In [13]:
len(table_loader.dataset)


47

In [35]:
test_sample = table_loader.dataset[2]
test_sample

{'id': 996,
 'table': {'id': '2-16369528-1.html.csv',
  'header': ['year',
   'best teenage / young adult',
   "reader 's vote",
   'best non - fiction',
   'lifetime achievement'],
  'rows': [['1996', 'na', 'na', 'na', 'jon cleary'],
   ['1997',
    'na',
    'na',
    'how to write crime edited by marele day',
    'alan yates (aka carter brown )'],
   ['1998', 'na', 'na', 'na', 'na'],
   ['1999', 'na', 'na', 'na', 'peter corris'],
   ['2000', 'na', 'na', 'na', 'na'],
   ['2001',
    'na',
    'bleeding hearts by lindy cameron',
    'na',
    'professor stephen knight'],
   ['2002',
    'blue murder by ken catran',
    'apartment 255 by bunty avieson',
    'na',
    'patrick gallagher'],
   ['2003', 'na', 'na', 'na', 'kerry greenwood'],
   ['2004', 'na', 'na', 'na', 'bob bottom'],
   ['2005', 'na', 'na', 'na', 'stuart coupe'],
   ['2006', 'na', 'na', 'na', 'andrew rule and john silvester'],
   ['2007', 'na', 'na', 'na', 'sandra harvey and lindsay simpson'],
   ['2008', 'na', 'na', 'na

### Data Augmentation

In [ ]:
#引入 schema(table measurement) statistical(numerical) enum string(char) date
#引入 term explanation（table comment(RAG)）
#引入 column summarization
#是否需要search engine
#table size, statistical features, header hierarchy


In [3]:
#schema 
from data_loader import TableFormat
from langchain.prompts.prompt import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI, OpenAI
model = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_base="https://api.chatanywhere.tech/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
# formatter = TableFormat(format='none', data=test_sample, use_sampling=True)
pre_instruction = PromptTemplate(input_variables=["table"], template=
"""
Instruction: Given the following table, you will add Metadata about the columns in the table.
Metadata includes:
- Numerical: whether the column content is numeric type like int or float.
- Char: whether the column content is a text or description.
- Date: whether the column content is datetime.

You need to output all the column names with metadata in angle brackets.
Example: name<Char>, launched<Date>, count<Numerical>

Table: {table}
Output:
""")
# 
# output = model.invoke([HumanMessage(content=pre_instruction.format(table=formatter.format_html()))])

In [16]:
output.response_metadata['token_usage']

{'completion_tokens': 1, 'prompt_tokens': 269, 'total_tokens': 270}

In [51]:
#summarization
pre_instruction_summary = PromptTemplate(input_variables=['table'], template=
"""
Instruction: Given the following table, you need to first summarize the contents of the table, then based on the summay, give a concluded description to each of the column.
Table: {table}

The output should use the following format: 
table summary: #summary for table contents
column description: You need to output all the column names with description in angle brackets
example: launched<The launched date for the competition> date<The date of the match>
""")
output = model.invoke([HumanMessage(content=pre_instruction_summary.format(table=formatter.format_html()))])

In [52]:
print(output.content)

table summary: The table shows the performance of a player in different tennis tournaments from 1986 to 1999.

column description: 
tournament<The name of the tennis tournament>
1986<The player's performance in the tournament in 1986>
1988<The player's performance in the tournament in 1988>
1989<The player's performance in the tournament in 1989>
1990<The player's performance in the tournament in 1990>
1991<The player's performance in the tournament in 1991>
1992<The player's performance in the tournament in 1992>
1993<The player's performance in the tournament in 1993>
1994<The player's performance in the tournament in 1994>
1995<The player's performance in the tournament in 1995>
1996<The player's performance in the tournament in 1996>
1997<The player's performance in the tournament in 1997>
1998<The player's performance in the tournament in 1998>
1999<The player's performance in the tournament in 1999>


In [11]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = "true"

In [10]:
#table_size
def get_table_size():
    return f'The table has {formatter.data.shape[0]} rows and {formatter.data.shape[1]} columns.'
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW", temperature=0.1)
from data_loader import TableAug,TableFormat
aug = TableAug(model)
# aug.schema_aug(test_sample)
test_sample = table_loader.dataset[0]
formatter = TableFormat(format='none')
output = aug.batch_summary_aug(formatter, table_loader.dataset[:2], batch_size=2, output_token=True)
# batch_data = table_loader.dataset[:2]

# print([formatter.load_data_from_dic({key: value[i] for key, value in batch_data.items()}).format_html() for i in range(len(batch_data.keys())) ])
# print([formatter.load_data_from_dic({key: value[i] for key, value in table_loader.dataset[:2].items()}).format_html() for i in range(2)])

KeyError: 0

In [9]:
print(output[0])

{'text': 'table summary: The table provides information about different wind farms including their names, scheduled dates, capacity in megawatts, number of turbines, types of turbines, and locations.\n\ncolumn description: \nwind farm<The name of the wind farm>\nscheduled<The scheduled date for the wind farm>\ncapacity (mw)<The capacity of the wind farm in megawatts>\nturbines<The number of turbines at the wind farm>\ntype<The type of turbines used at the wind farm>\nlocation<The location of the wind farm>'}


In [8]:
print(formatter.format_nl_sep())

Col :Name|Title|Date_c_from|Date_until|Russian_state|Austrian_state
Row 1 :Lev Pavlovich Urusov|Ambassador|1905|1910|Russian Empire|Austria-Hungary
Row 2 :Konstantin Konstantinovich Yurenev|Plenipotentiary|1 October 1927|24 January 1933|Soviet Union|Federal State of Austria
Row 3 :Averky Borisovich Aristov|Ambassador Extraordinary and Plenipotentiary|20 September 1971|11 July 1973|Soviet Union|Republic of Austria


## Embedding

In [84]:
sample['table']['id']

'csv/204-csv/925.tsv'

In [131]:
import pandas as pd
from utils import parse_specific_composition, parse_output
sample = table_loader.normalize_table(table_loader.dataset[505])
formatter = TableFormat(format='none',data=sample)
formatter.all_data
row_string = []
task_name = 'wikitable'
split = 'test'
schema_information = pd.read_csv(f"result/aug/{task_name}_{split}_schema.csv", index_col='table_id')
aug_information = pd.read_csv(f"result/aug/{task_name}_{split}_summary.csv", index_col='table_id')
composition_information = pd.read_csv(f"result/aug/{task_name}_{split}_composition.csv", index_col='table_id')
summary_aug, column_aug = aug_information.loc[sample['table']['id']]['summary'], aug_information.loc[sample['table']['id']]['column_description']
col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
extra_col_info = []
for i_c in range(len(col_names)):
    extra_col_info.append(f'{col_names[i_c]}: {col_infos[i_c]}')
# extra_information = '\n'.join(parse_specific_composition(composition_information.loc[sample['table']['id']]['composition'], formatter.all_data.columns))
# for i in range(len(formatter.all_data.columns)):
#     row_string.append(formatter.all_data.columns[i])
# from langchain_text_splitters import CharacterTextSplitter
# text_splitter = CharacterTextSplitter(chunk_size=1, chunk_overlap=0, separator='\n\n')
# texts = text_splitter.split_text('\n\n'.join(row_string))

In [15]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_base="https://api.chatanywhere.com.cn/v1", openai_api_key="sk-WZtqZEeuE0Xb6syVghDgAxdwe0ASWLkQRGxl61UI7B9RqNC4")

In [91]:
sample['query']

'which team had the first pick this round?'

In [132]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_texts(extra_col_info, embeddings)

In [185]:
retriever = db.as_retriever(search_kwargs={"include_metadata": True, "score_threshold": 0.4})
result = retriever.invoke(sample['query'])

In [140]:
[r.page_content.split(':')[0].strip() for r in result]  + ['a']

['NHL_team', 'Pick_num', 'a']

In [139]:
formatter.all_data[[r.page_content.split(':')[0].strip() for r in result]]

,NHL_team,Pick_num
0,Winnipeg Jets,148
1,Detroit Red Wings,149
2,Colorado Rockies,150
3,Hartford Whalers,151
4,Washington Capitals,152
5,Toronto Maple Leafs,153
6,Pittsburgh Penguins,154
7,Edmonton Oilers,155
8,New York Rangers,156
9,Vancouver Canucks,157


In [100]:
import re
pattern = re.compile(r'# Row (\d+)')
rowinds = [int(pattern.search(r.page_content).group(1)) for r in result]

AttributeError: 'NoneType' object has no attribute 'group'

In [41]:
formatter.all_data.loc[rowinds]

,Name,League,FA_Cup,League_Cup,JP_Trophy,Total
8,Jamie Cureton,20,0,0,0,20
5,Guillem Bauza,2,0,0,0,2
7,Pat Baldwin,1,0,0,0,1
11,OWN GOALS,0,0,0,0,0


### Few Shot learning

In [9]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
summary_examples = ['The columns in the table are "wind farm, scheduled, capacity (mw), turbines, type, and location." The rows in the table represent different wind farms, with information about their scheduled dates, capacity, number of turbines, type, and location.',
                    "The table provides information about different destroyers, including their kanji (Japanese characters), names, builders, dates when they were laid down, launched, and completed."]
subtable_examples = ['Columns(wind farm, scheduled, turbines), Rows(12)',
                    'Columns(kanji, builder), Rows(ALL)']
inds = [0, 100]
examples_prompt = PromptTemplate(input_variables=["table", "claim", "summary", "subtable"], template=
"""
Table: {table}
Claim: {claim}
Summary: {summary}
Subtable: {subtable}""")
formatter = TableFormat(format='none', data=test_sample, use_sampling=True)
num_k = 2
# examples = [examples_prompt.format(**{"table": TableFormat(format='none', data=table_loader.dataset[inds[i]]).format_html(),
#                                     "claim": table_loader.dataset[inds[i]]['statement'],
#                                     "summary": summary_examples[i],
#                                     "subtable": subtable_examples[i]}) for i in range(num_k)]

examples_dict = [{"table": TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_html(),
                                    "claim": table_loader.dataset[inds[i]]['statement'],
                                    "summary": summary_examples[i],
                                    "subtable": subtable_examples[i]} for i in range(num_k)]
prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="Instruction: Given the following table and claim, let's first summarize the contents of the rows and columns of the table, and then select relevent rows/columns in the given table that support or oppose the statement.",
    suffix=
    """
Table: {table}
Claim: {claim}
    """,
    input_variables=["table", "claim"],
)
print(prompt.format(table=formatter.format_html(), claim=test_sample['statement']))

Instruction: Given the following table and claim, let's first summarize the contents of the rows and columns of the table, and then select relevent rows/columns in the given table that support or oppose the statement.


Table: <table>
<thead>
<tr><th>  wind farm</th><th>  scheduled</th><th>  capacity (mw)</th><th>  turbines</th><th>              type</th><th>      location</th></tr>
</thead>
<tbody>
<tr><td>codling    </td><td>unknown    </td><td>1100           </td><td>220       </td><td>unknown           </td><td>county wicklow</td></tr>
<tr><td>carrowleagh</td><td>2012       </td><td>36.8           </td><td>16        </td><td>enercon e - 70 2.3</td><td>county cork   </td></tr>
<tr><td>gortahile  </td><td>2010 autumn</td><td>20             </td><td>8         </td><td>nordex n90        </td><td>county laois  </td></tr>
</tbody>
</table>
Claim: the scheduled date for the farm with 17 turbine be 2012
Summary: The columns in the table are "wind farm, scheduled, capacity (mw), turbines, t

In [5]:
from data_loader import TableFormat
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.messages import HumanMessage
from langchain.prompts.prompt import PromptTemplate
model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
model.invoke([HumanMessage(content=prompt.format(table=formatter.format_html(), claim=test_sample['statement']))])

AIMessage(content='Summary: The table provides information about different events, including their names, establishment years, categories, subcategories, and main venues.\nSubtable: Columns(event name, established, category), Rows(2)', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 931, 'total_tokens': 972}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None})

### Few-shot for operations

In [38]:
from data_loader import TableFormat
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.messages import HumanMessage
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
model = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
formatter = TableFormat(format='none', data=test_sample, use_sampling=True)
inds = [0,2]
summary_examples = ['The columns in the table are "wind farm, scheduled, capacity (mw), turbines, type, and location." The rows in the table represent different wind farms, with information about their scheduled dates, capacity, number of turbines, type, and location.',
                    "The table provides information about different events, including their names, establishment years, categories, subcategories, and main venues."]
Output_examples = ['wind farm<DELETE>, scheduled<KEEP>, capacity (mw)<DELETE>, turbines<KEEP>, type<KEEP>, location<KEEP>', 'event name<KEEP> established<KEEP> category<KEEP> sub category<DELETE> main venue<DELETE>']
examples_prompt = PromptTemplate(input_variables=["table", "claim", "output"], template=
"""
Table: {table}
Claim: {claim}
Output: {output}""")
num_k = 2
examples_dict = [{"table": TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_html(),
                                    "claim": table_loader.dataset[inds[i]]['statement'],
                                    # "summary": summary_examples[i],
                                    "output": Output_examples[i]} for i in range(num_k)]
prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
Instruction: Given the following table and claim, you will output the operations corresponding to each column which can help us judging the truth or falsity of claim.
Operations: DELETE(delete column unrelevant to the claim), KEEP(keep column relevant to the claim), GROUP BY(combine aggregate functions and group the result set by one or more columns), COUNT(returns the number of rows in column), AVG(returns the average value of a numeric column), SUM(returns the sum of a numeric column), MAX(returns the max value of a numeric column), MIN(returns the min value of a numeric column), ORDER BY(sort the value in ascending order)""",
    suffix=
    """
Table: {table}
Thought: your reason here
Claim: {claim}
    """,
    input_variables=["table", "claim"],
)
print(prompt.format(table=formatter.format_html(), claim=test_sample['statement']))

# output = model.invoke([HumanMessage(content=prompt.format(table=formatter.format_html(), claim=test_sample['statement']))])



You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
Instruction: Given the following table and claim, you will output the operations corresponding to each column which can help us judging the truth or falsity of claim.
Operations: DELETE(delete column unrelevant to the claim), KEEP(keep column relevant to the claim), GROUP BY(combine aggregate functions and group the result set by one or more columns), COUNT(returns the number of rows in column), AVG(returns the average value of a numeric column), SUM(returns the sum of a numeric column), MAX(returns the max value of a numeric column), MIN(returns the min value of a numeric column), ORDER BY(sort the value in ascending order)


Table: <table>
<thead>
<tr><th>  wind farm</th><th>  scheduled</th><th>  capacity (mw)</th><th>  turbines</th><th>              type</th><th>      location</th></tr>
<

In [39]:
output = model.invoke([HumanMessage(content=prompt.format(table=formatter.format_html(), claim=test_sample['statement']))])
print(output)

content='Output: date<DELETE>, tournament<DELETE>, surface<DELETE>, partner<KEEP>, opponents<KEEP>, score<DELETE>' response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 1037, 'total_tokens': 1063}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}


In [35]:
print(formatter.format_psql())
print(test_sample['statement'])
print(test_sample['label'])

+------------------+-----------------------------------+-----------+---------------------+------------------------------------------------+-----------------------+
| date             | tournament                        | surface   | partner             | opponents                                      | score                 |
|------------------+-----------------------------------+-----------+---------------------+------------------------------------------------+-----------------------|
| 15 january 2007  | algiers 2 , algeria               | clay      | rushmi chakravarthi | barbora matusova anna savitskaya               | 6 - 2 , 6 - 0         |
| 11 february 2008 | mallorca 2 , spain                | clay      | stephanie vogt      | leticia costas - moreira maite gabarrus alonso | 7 - 6 (7 - 2) , 6 - 3 |
| 28 april 2008    | makarska , croatia                | clay      | stephanie vogt      | tadeja majerić maša zec peškirič               | 7 - 5 , 6 - 2         |
| 8 september 20

## Zero-shot learning


In [22]:
from tabulate import tabulate
formatter = TableFormat(format='none', data=test_sample)
print(test_sample['statement'])
print(tabulate(formatter.data, headers=formatter.data.columns, tablefmt='psql'))
# print('Summary: The table provides information about different events, including their names, establishment years, categories, subcategories, and main venues.\nSubtable: Columns(event name, established, category), Rows(2)')

uraga dock company produce the most destroyer on the list , at 3
+----+---------+-------------------+--------------------------------+------------------+------------------+-------------------+
|    | kanji   | name              | builder                        | laid down        | launched         | completed         |
|----+---------+-------------------+--------------------------------+------------------+------------------+-------------------|
|  0 | 睦月    | mutsuki dd - 19   | sasebo naval arsenal , japan   | 21 may 1924      | 23 july 1925     | 25 march 1926     |
|  1 | 如月    | kisaragi dd - 21  | maizuru naval arsenal , japan  | 3 june 1924      | 5 june 1925      | 21 december 1925  |
|  2 | 彌生    | yayoi dd - 23     | uraga dock company , japan     | 11 january 1924  | 11 july 1925     | 28 august 1926    |
|  3 | 卯月    | uzuki dd - 25     | ishikawajima shipyards , japan | 11 january 1924  | 15 october 1925  | 14 september 1926 |
|  4 | 皐月    | satsuki dd - 27   | fujinagata s

In [10]:
from data_loader import TableFormat, TableLoader
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.messages import HumanMessage
from langchain.prompts.prompt import PromptTemplate
model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
table_loader = TableLoader(table_name='wikitable', split='test', use_sample=True, small_test=False)
test_sample = table_loader.dataset[40]
formatter = TableFormat(format='none', data=test_sample, use_sampling=True)
pre_instruction_schema = PromptTemplate(input_variables=["table"], template="""
Instruction: Given the following table, you need to summarize the contents of the table and tell what table is about.
Table: {table}

The output should use the following format: 
Summary: #summary for table contents

Summary:
""")
output = model.invoke([HumanMessage(content=pre_instruction_schema.format(table=formatter.format_html(),))])



In [11]:
print(output.content)

The table provides information about different games played by a football team. It includes details like the week number, date of the game, opponent team, game result, game site, NFL recap, and the attendance for each game.


In [40]:
from tabulate import tabulate
print(tabulate(formatter.data, headers=formatter.data.columns, tablefmt='psql'))
print('Summary: The table provides information about different events, including their names, establishment years, categories, subcategories, and main venues.\nSubtable: Columns(event name, established, category), Rows(touchdown atlantic)')

+----+--------------------------------------------+---------------+----------------+----------------+--------------------------+
|    | event name                                 |   established | category       | sub category   | main venue               |
|----+--------------------------------------------+---------------+----------------+----------------+--------------------------|
|  0 | dieppe kite international                  |          2001 | sporting       | kite flying    | dover park               |
|  1 | the frye festival                          |          2000 | arts           | literary       | university of moncton    |
|  2 | hubcap comedy festival                     |          2000 | arts           | comedy         | various                  |
|  3 | touchdown atlantic                         |          2010 | sporting       | football       | moncton stadium          |
|  4 | atlantic nationals automotive extravaganza |          2000 | transportation | automotive  

In [41]:
print(test_sample['statement'])

touchdown atlantic , in the category of sporting , be establish in 2010


In [38]:
import os
from tenacity import retry, stop_after_attempt, wait_random_exponential
from typing import List
from openai import OpenAI
class Config:
    def __init__(self) -> None:
        self.openai_api_key = "sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW"
        self.base_url = "https://api.chatanywhere.cn/v1"
        self.model = "gpt-3.5-turbo"

class CallLLM:
    """Class for calling the OpenAI Language Model API."""

    def __init__(self, config: Config):
        self.client = OpenAI(api_key=config.openai_api_key,
                             base_url=config.base_url)
        self.model = config.model
        
    @retry(wait=wait_random_exponential(min=30, max=60), stop=stop_after_attempt(1000))
    def generate_text(self, prompt: List[str]) -> List[str]:
        """Generate text based on the prompt and instruction."""

            # batched examples, with xx completions per request
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}], 
            temperature=0,
            max_tokens=96,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0,
        )
        # match completions to prompts by index
        return response.choices[0].message.content.strip()
configs = Config()  
llm = CallLLM(config=configs)


In [39]:

input_text = "\n".join(input)
llm.generate_text(input_text)

'1'

In [15]:
import pandas as pd
def process_input(sample, instruction):
    df = pd.DataFrame(columns=sample["table"]["header"])
    for i, line in enumerate(sample['table']['rows']):
        df.loc[i] = line
        
    texts = [instruction,
          "The database table DF is shown as follows: \n",
          df.to_html(),
          "query:",
          sample['statement'],
           "Output the code braced by '```'. \n SQL:"]
    
    label = sample['label']
    return "\n".join(texts), label 
    

In [4]:
subtable_instruction = """
You are a SQLite expert. Given an input query, identify critical values and ranges of the table, then create a syntactically correct SQLite query to create a VIEW. To create a syntactically correct SQL view, the selected data within this view must be helpful in answering the question. During the construction of the view, if column names are confusing, rename the columns accordingly.
"""

In [16]:
print(process_input(test_sample, subtable_instruction))

('\nYou are a SQLite expert. Given an input query, identify critical values and ranges of the table, then create a syntactically correct SQLite query to create a VIEW. To create a syntactically correct SQL view, the selected data within this view must be helpful in answering the question. During the construction of the view, if column names are confusing, rename the columns accordingly.\n\nThe database table DF is shown as follows: \n\n<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>date</th>\n      <th>result</th>\n      <th>score</th>\n      <th>brazil scorers</th>\n      <th>competition</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>may 11 , 1919</td>\n      <td>w</td>\n      <td>6 - 0</td>\n      <td>friedenreich (3) , neco (2) , haroldo</td>\n      <td>south american championship</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>may 18 , 1919</td>\n      <td>w</td>\n      <td>3 - 1

### Simple pipeline

In [6]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:///db/sqlite/tabfact.db')
normalized = table_loader.normalize_table(test_sample)
df = pd.DataFrame(columns=normalized['table']['header'])
for ind, r in enumerate(normalized['table']['rows']):
    df.loc[ind] = r
# print(' '.join(normalized['table']['header']) + '*************')
df.to_sql(name='ind100', con=engine, if_exists='replace', index=False)
# table_loader.table2db(engine, test_sample)

12

In [34]:
from tabulate import tabulate
structured_data_markdown = tabulate(
            df, headers=df.columns, tablefmt="pipe", showindex=True
        )

In [35]:
structured_data_markdown

'|    | date          | result   | score   | brazil scorers                        | competition                 |\n|---:|:--------------|:---------|:--------|:--------------------------------------|:----------------------------|\n|  0 | may 11 , 1919 | w        | 6 - 0   | friedenreich (3) , neco (2) , haroldo | south american championship |\n|  1 | may 18 , 1919 | w        | 3 - 1   | heitor , amílcar , millon             | south american championship |\n|  2 | may 26 , 1919 | d        | 2 - 2   | neco (2)                              | south american championship |\n|  3 | may 29 , 1919 | w        | 1 - 0   | friedenreich                          | south american championship |\n|  4 | june 1 , 1919 | d        | 3 - 3   | haroldo , arlindo (2)                 | taça roberto cherry         |'

In [1]:

from langchain_openai import ChatOpenAI, OpenAI
model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
model = ChatOpenAI(model_name="gpt-4", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")

In [7]:
import re
def extract_yes_no_and_map(text):
    # Convert the input text to lowercase for case-insensitive matching
    text = text.lower()

    # Define regular expressions for yes/no matching
    yes_patterns = [r'\byes\b', r'\btrue\b']
    no_patterns = [r'\bno\b', r'\bfalse\b']

    # Check for "0"
    if text == "0":
        return "0"

    # Check for "1"
    if text == "1":
        return "1"

    # Check for yes
    for pattern in yes_patterns:
        if re.search(pattern, text):
            return "1"

    # Check for no
    for pattern in no_patterns:
        if re.search(pattern, text):
            return "0"

    # Return 2 if neither yes nor no is found
    return "2"

def eval_fv_match(pred_list, gold_list):
        acc = 0.0
        for pred, gold in zip(pred_list, gold_list):
            pred, gold = extract_yes_no_and_map(pred), extract_yes_no_and_map(gold)
            if pred == gold:
                acc += 1
        acc = acc / len(pred_list)
        return acc
    
def process_input(sample, instruction):
    df = pd.DataFrame(columns=sample["table"]["header"])
    for i, line in enumerate(sample['table']['rows']):
        df.loc[i] = line
    texts = [instruction,
          "the table needed to be answered: \n",
          df.to_html(),
          "query:",
          sample['statement'],
           "answer is: \n"]
    
    label = sample['label']
    return "\n".join(texts), label 

In [37]:
#简易版本的pipeline
instruction = "Read the table below to verify whether the provided claim/query are true or false. Return 0 if it's false, or 1 if it's true. Only return 0 or 1 without any other information. \n"
num_samples = 32
batch_size = 32
num_batches = num_samples // batch_size
batches = []
for batch_num in range(num_batches):
    batch_prompt, ground = [], []
    start = batch_num * batch_size
    batch_data = tabfact['validation'][start: start+batch_size]
    for i in range(batch_size):
        prompt, label = process_input(sample=dict({key: value[i] for key, value in batch_data.items()}), instruction=instruction)
        batch_prompt.append(prompt)
        ground.append(str(label))
        # call llm
    batch_pred = list(map(lambda x: x.content, model.batch(batch_prompt)))
        
        #do evaluation
    accuracy = eval_fv_match(batch_pred, ground)
    print(accuracy)
        
        
        
    

        
        

0.6875


In [42]:
def display_wrong_result(pred_list, gold_list, dataset):
    for ind, (pred, gold) in enumerate(zip(pred_list, gold_list)):
        pred, gold = extract_yes_no_and_map(pred), extract_yes_no_and_map(gold)
        if pred != gold:
            sample= dataset[ind]
            df = pd.DataFrame(columns=sample["table"]["header"])
            for i, line in enumerate(sample['table']['rows']):
                df.loc[i] = line
            print(sample['statement'], sample['label'])
            print(tabulate(df, headers=df.columns, tablefmt='psql'))
        
print(display_wrong_result(batch_pred, ground, tabfact['validation']))

süper lig be the most common league to win a round in the turkish cup 1
+----+---------------+-------------------+------------------+-------------------------------+--------------------------+----------------------------------------------------+
|    | round         |   clubs remaining |   clubs involved | winners from previous round   | new entries this round   | leagues entering at this round                     |
|----+---------------+-------------------+------------------+-------------------------------+--------------------------+----------------------------------------------------|
|  0 | first round   |               156 |               86 | none                          | 86                       | tff third league & turkish regional amateur league |
|  1 | second round  |               113 |              108 | 43                            | 65                       | süper lig & tff first league & tff second league   |
|  2 | third round   |                59 |               5

In [40]:
for ind, i in enumerate(zip(batch_pred, ground)):
    print(ind, i)

0 ('1', '1')
1 ('1', '1')
2 ('1', '1')
3 ('1', '1')
4 ('1', '1')
5 ('1', '0')
6 ('0', '0')
7 ('0', '0')
8 ('0', '0')
9 ('0', '0')
10 ('1', '1')
11 ('0', '1')
12 ('1', '1')
13 ('1', '1')
14 ('0', '1')
15 ('0', '0')
16 ('0', '0')
17 ('0', '0')
18 ('1', '0')
19 ('1', '0')
20 ('The answer is 1.', '1')
21 ('0', '1')
22 ('1', '1')
23 ('0', '1')
24 ('1', '1')
25 ('0', '0')
26 ('0', '0')
27 ('1', '0')
28 ('0', '0')
29 ('0', '0')
30 ('1', '1')
31 ('1', '1')


## Answer output

In [6]:

from data_loader import TableFormat
import pandas as pd
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.messages import HumanMessage
from langchain.prompts.prompt import PromptTemplate
from data_loader import TableLoader

table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
from utils import normalize_schema
model = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_base="https://api.chatanywhere.tech/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
normalized_sample = table_loader.normalize_table(
                    table_loader.dataset[3])
formatter = TableFormat(format='none', data=normalized_sample)
# schema_information = pd.read_csv(f"result/aug/tabfact_test_schema.csv", index_col='table_id')
# formatter.data = normalize_schema(formatter.data, schema_information.loc[normalized_sample['id']]['schema'])
pre_instruction = PromptTemplate(input_variables=["table"], template=
"""
Below is a subtable with rows sampled, you are required to infer the data distribution and format from the sample data.
Refine commonalities about the structure within each table column.
You need to output in the following format: 
number. Column_name: Commonalities
#example format
1. championship: Names of golf tournaments are listed with some additional information (e.g., 's open, classic)

sub-table: {table}
""")
# 
output = model.invoke([HumanMessage(content=pre_instruction.format(table=formatter.format_html()) )])



In [7]:
print(output.content)

1. position: Integer values representing the team's position in the table
2. team: Names of football teams
3. played: Number of games played by each team
4. drawn: Number of games drawn by each team
5. lost: Number of games lost by each team
6. goals_for: Number of goals scored by each team
7. goals_against: Number of goals conceded by each team
8. goal_difference: Goal difference calculated as goals_for - goals_against
9. points_1: Total points earned by each team in the league


In [39]:
print(output.content)

1. game column: It appears to represent the game number, with each entry being a numerical value.
2. date column: It represents the date of the game, with each entry being in a specific date format (e.g., "month day").
3. team column: It represents the name of the team, with each entry being a text string.
4. score column: It represents the game score, with each entry showing the score in a specific format (e.g., "l 90 - 115 (ot)").
5. high_points column: It represents the player with the highest points in the game, with each entry showing the player's name and the number of points.
6. high_rebounds column: It represents the player with the highest rebounds in the game, with each entry showing the player's name and the number of rebounds.
7. high_assists column: It represents the player with the highest assists in the game, with each entry showing the player's name and the number of assists.
8. location_attendance column: It represents the location of the game and the attendance, with 

In [8]:
from data_loader import TableLoader, TableFormat
inputs = []
row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is a subtable with columns filtered, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information, write a SQLITE3 SELECT SQL statement using table DF that complete query.
sub-table: {table}
Query: {claim}
Extra information: {aug}
SQL: """)
inputs.append(dict({'table': formatter.format_html(table_caption=normalized_sample['table']['caption']),
                                        'claim': normalized_sample['query'],
                                        # 'claim':"who is the winner of the lifetime achievement award after 2005?",
                                        'aug':  output.content
                                        }))

In [11]:
from data_loader import TableLoader, TableFormat
inputs = []
row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is a subtable with columns filtered, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information, write a SQLITE3 SELECT SQL statement using table DF that provide clues for query.
sub-table: {table}
Query: verify whether the provided claim/query are true or false. {claim}
Extra information: {aug}
SQL: """)
inputs.append(dict({'table': formatter.format_html(table_caption=normalized_sample['table']['caption']),
                                        'claim': normalized_sample['query'],
                                        # 'claim':"who is the winner of the lifetime achievement award after 2005?",
                                        'aug':  output.content
                                        }))
# verify whether the provided claim/query are true or false.

In [35]:
from data_loader import TableLoader, TableFormat
inputs = []

row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is a subtable with columns filtered, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information and data in subtable, write 3 SELECT SQL statements using table DF that complete or related to query.
The SQL may select different useful parts for the query.
sub-table: {table}
Query: {claim}
Extra information: {aug}
SQL: """)
inputs.append(dict({'table': formatter.format_html(table_caption=normalized_sample['table']['caption']),
                                        'claim': normalized_sample['query'] + ". The query is about 2005 Milwaukee Brewers season red uniforms vs Padres schedule",
                                        'aug':  output.content
                                        }))

In [9]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
model = ChatOpenAI(model_name='gpt-3.5-turbo-0125', openai_api_base="https://api.chatanywhere.tech/v1",
                       openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW", temperature=0.1)
llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=True)
batch_pred = llm_chain.batch(inputs, return_only_outputs=True)



> Entering new LLMChain chain...
Prompt after formatting:

Our ultimate goal is to answer query based on the table. Below is a subtable with columns filtered, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information, write a SQLITE3 SELECT SQL statement using table DF that complete query.
sub-table: <table>
<caption>1986 - 87 north west counties football league</caption>
<thead>
<tr><th>  position</th><th>               team</th><th>  played</th><th>  drawn</th><th>  lost</th><th>  goals_for</th><th>  goals_against</th><th>  goal_difference</th><th>  points_1</th></tr>
</thead>
<tbody>
<tr><td>12        </td><td>ashton town        </td><td>24      </td><td>9      </td><td>11    </td><td>29         </td><td>39             </td><td>10               </td><td>15 2      </td></tr>
<tr><td>10        </td><td>padiham            </td><td>24      </td><td>8      </td><td>12    </td><td>

In [10]:
print(batch_pred[0]['text'])

SELECT team
FROM DF
WHERE goals_for = (SELECT MAX(goals_for) FROM DF)
AND caption = '1986 - 87 north west counties football league'
AND team = 'flixton';


In [19]:
from data_loader import TableLoader, TableFormat
inputs = []

pre_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is relevant information regrading the table stucture. Carefully analyze the query, based on the augmentation information you are required to infer the data distribution and format. Write an SELECT SQL statement using table DF that is most likely to provide useful information to the query.
Query: {claim}
Extra information: {aug}
SQL: """)
inputs.append(dict({
                                        'claim': normalized_sample['query'],
                                        'aug':  output.content
                                        }))

In [22]:
from data_loader import TableLoader, TableFormat
inputs = []

row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is an initial SQL statement about the query. Now you can see some smaple rows from the table, improve the sql statement so that the SQL can answer the query.
initial SQL: {initial}
sub-table: {table}
Query: {claim}

SQL: """)
inputs.append(dict({"initial" : initial[0]['text'],
                                        'claim': normalized_sample['query'],
                'table': formatter.format_html(table_caption=normalized_sample['table']['caption']),
                                        }))

In [18]:
from data_loader import TableFormat
from executor import SQLManager
from sqlalchemy import create_engine
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine)
i = 0
# SQL = manager.format_sql(preds[i]['pred'].split(':')[1])


def show_table(data):
    formatter = TableFormat(format='none', data=data)
    columns = [c.strip() for c in preds[i]['pred'].split(',')]
    print(data['statement'])
    formatter.data = formatter.data.loc[:, columns]
    
    print(formatter.format_html())
    print(formatter.format_psql())
    print(data['label'])
    # data.columns = [manager.normalize_col_name(c) for c in formatter.all_data.columns]
    # data.to_sql('DF', manager.engine, if_exists='replace', index=False)
    
    # subtable = pd.read_sql(command, self.engine)
    # test_df = manager.execute_from_df(SQL, formatter.all_data)
    # return test_df
test_df = show_table(table_loader.dataset[i])
print(test_df)

the 31997 mintage be release in 2001
<table>
<thead>
<tr><th>  year</th><th>                  mintage</th></tr>
</thead>
<tbody>
<tr><td>2003  </td><td>included in hmcs bras dor</td></tr>
<tr><td>2003  </td><td>31997                    </td></tr>
<tr><td>2000  </td><td>44367                    </td></tr>
</tbody>
</table>
+--------+----------------------------+-------------------+-----------------------------+---------------+
|   year | theme                      | artist            | mintage                     |   issue price |
|--------+----------------------------+-------------------+-----------------------------+---------------|
|   2000 | steam buggy                | john mardon       | 44367                       |         59.95 |
|   2000 | the bluenose               | j franklin wright | included in steam buggy     |         59.95 |
|   2000 | the toronto                | john mardon       | included in steam buggy     |         59.95 |
|   2001 | the russell light four     | 

In [ ]:
# Answer output
pre_instruction = PromptTemplate(input_variables=["table", "claim"], template=
"""
Our ultimate goal is to answer query based on the table. Below is a SQL statement which reports error or returns no result. You need to understand the logic behind the SQL filtering and rewrite SQL to guarantee the SQL return useful information. 
sub-table: {table}
SQL Excuted: 
```{SQL}```
Query: {claim}
""")
output = model.invoke([HumanMessage(content=pre_instruction.format(table=formatter.format_html(), claim=test_sample['statement']))])

## Evaluat from local result

In [1]:
import json
from data_loader import TableLoader
table_loader = TableLoader(table_name='tabfact', split='test', small_test=True, use_sample=False)
preds = []
with open('./result/answer/tabfact_test_04-20_02-06-16.json', 'r') as f:
    lines = f.readlines()
    for l in lines:
        preds.append(json.loads(l)['pred'])

print(len(preds))
SQLs = []
with open('./result/SQL/tabfact_test_04-20_02-04-20.json', 'r') as f:
    lines = f.readlines()
    for l in lines:
        SQLs.append(json.loads(l)['pred'])
print(len(SQLs))
# do evaluation
# accuracy = eval_fv_match(pred_label, labels)



255
255


In [3]:
from utils import eval_tabfact
import json
from data_loader import TableLoader
import numpy as np
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
labels = [table_loader.normalize_table(line)['label'] for line in table_loader.dataset][:255]
answers = eval_tabfact('./result/answer/tabfact_test_04-20_02-06-16.json', labels, verbose=True)
print(answers)

['0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '0', '0', '0', '0', '1', '0', '1', '1', '0', '1', '0', '0', '1', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '1', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '1', '0', '0', '1', '0', '1', '1', '0', '0', '1', '1', '0', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '1', '1', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '1', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '0', '0', '0', '0', '1', '0', '1',

In [4]:
from utils import eval_tabfact
import json
from data_loader import TableLoader
import numpy as np
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=True, small_test=True)
labels = [table_loader.normalize_table(line)['label'] for line in table_loader.dataset]
answers = eval_tabfact('./result/answer/tabfact_test_04-18_15-20-27.json', labels, verbose=True)
print(np.where(np.array(labels) != np.array(answers)))

['0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '0', '1', '0', '1', '1', '0', '1', '1', '0', '0', '0', '1', '1', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '1', '1', '1', '1', '0']
(array([ 2,  3, 14, 21, 25, 27, 32, 35]),)


In [39]:
data = table_loader.dataset[i]
df = pd.DataFrame(columns=data["table"]["header"])
for i, line in enumerate(data['table']['rows']):
    df.loc[i] = line
data = df

In [40]:
formatter = TableFormat(format='none', data=table_loader.dataset[i])
from functools import partial
data_func = partial(pd.to_datetime, dayfirst=True, format='mixed')
formatter.all_data['date'] = data_func(formatter.all_data['date'])
formatter.all_data['date'] = pd.to_datetime(formatter.all_data['date'], format='%Y-%m')
formatter.all_data['date'] = formatter.all_data['date'].dt.strftime('%Y-%m-%d')

In [72]:
import pandas as pd
formatter = TableFormat(format='none', data=table_loader.dataset[i])
schema_information = pd.read_csv(f"result/aug/tabfact_test_schema.csv", index_col='table_id')
formatter.normalize_schema(schema_information.loc[table_loader.dataset[i]['table']['id']]['schema'])
formatter.all_data.to_sql('DF', con=engine, if_exists='replace', index=False)

27

In [2]:
inds = [0,   1,  11,  14,  22,  24,  26,  31,  37,  38,  39,  40,  47,
       49,  50,  59,  62,  63,  65,  66,  68,  69,  70,  75,  80,  81,
       82,  90,  91,  93,  95,  99, 100, 105, 106, 108, 110, 116, 119,
124, 129, 136, 137, 145, 147, 148, 151, 153, 155, 158, 160, 164,
167, 169, 171, 173, 175, 179, 180, 181, 187, 188, 190, 192, 198,
200, 207, 209, 210, 211, 217, 225, 227, 230, 233, 236, 237, 241,
251]
len(inds)

79

#### show tabfact

In [1]:
import sys
sys.path.append('/media/disk1/chatgpt/zh/tabular_data')

In [32]:
********210***************
1 0
Thought: Based on the SQL query provided, Lleyton Hewitt was a runner-up 5 times. The total number of times he was a runner-up is 5, and the total number of times he participated is not given. 
Answer: 0
['how many times was Lleyton Hewitt a runner-up in total?', ' How many times did Lleyton Hewitt be a runner-up?', 'How many times did Lleyton Hewitt win a championship?']
********213***************
0 1
Thought: Based on the SQL query provided, there were no instances of a master series final where one side finished with 4 points. 
Answer: 1
["How many of Lleyton Hewitt's master series final have one side finishing with 4 points?", "How many of Lleyton Hewitt's master series finals ended with one side finishing with 4 points?", " How many of Lleyton Hewitt's master series final have the other side finishing with 4 points?"]
********225***************
0 1
Thought: Based on the sub-table, it appears that no team scored in the first game of the World Cup in France on June 19, 1998.
Answer: 1
['What was the score for the first game of the world cup in France?', 'which team did not score in the first game of the world cup in France?', ' Did any team score for the first game of the world cup in France?']
********227***************
0 1
Thought: Based on the SQL query provided, the Solheim Cup was hosted in the US 7 times from 1990 through 2013, as indicated by the sub-table.
Answer: 1
[' When was the Solheim Cup hosted in the US from 1990 through 2013?', 'How many times was the Solheim Cup hosted in the US from 1990 through 2013?', 'how many times was the Solheim Cup hosted in the US from 1990 through 2013?']
********228***************
1 0
Thought: Based on the SQL query provided, the United States team won 8 Solheim Cup tournaments between 1990 and 2009. The last win for the United States team before 2009 was in 2007.
Answer: 0
['How many Solheim Cup tournaments did the United States team win from 1990 to 2009?', 'how many Solheim Cup tournaments did the United States team win between 1990 and 2009?', ' When was the last Solheim Cup win for the United States team before 2009?']
********229***************
1 0
Thought: Based on the SQL query provided, the captains who have won the cup multiple times are Mickey Walker and Kathy Whitworth. Patty Sheehan, Judy Rankin, and Kathy Whitworth are not the only captains who have won the cup 2 times.
Answer: 0
['How many times did Patty Sheehan win the cup as captain?', ' How many times did Judy Rankin win the cup as captain?', ' How many times did Kathy Whitworth win the cup as captain?', 'which captains have won the cup multiple times?']
********234***************
0 1
Thought: Based on the sub-table, Patty Sheehan, Judy Rankin, and Kathy Whitworth all captained a winning team in the Solheim Cup 1 time.
Answer: 1
['How many times did Patty Sheehan captain a winning team in the Solheim Cup?', 'how many times did each captain win the solheim cup?', ' How many times did Judy Rankin captain a winning team in the Solheim Cup?', ' How many times did Kathy Whitworth captain a winning team in the Solheim Cup?']
********239***************
1 0
Thought: Based on the SQL query provided, the attendance on August 26 was 48063 and the attendance on August 27 was 48032.
Answer: 0
[' What was the attendance on August 27?', 'What was the attendance on August 26?', 'what was the attendance on August 26 and August 27?']
********240***************
1 0
Thought: Based on the SQL query provided, when the score was 10 - 4 in a game against the Athletics, the Colorado Rockies' record was 68 - 70.
Answer: 0
["When the score be 10 - 4 , what was the Colorado Rockies' record in a game against the athletics?", "what is the Colorado Rockies' record after each game?"]
********254***************
0 1
Thought: Based on the SQL query provided, the smallest attendance was at the game against the Baltimore Colts, with an attendance of 50451. 
Answer: 1
['which game had the smallest attendance?', 'What was the attendance at the game against the New England Patriots?', ' What was the attendance at the game against the Baltimore Colts?']
********255***************
0 1
Thought: Based on the SQL query provided, the attendance in week 12 was 66875, which is higher than the attendance in week 6 (71009) and week 13 (43475). 
Answer: 1
[' What was the attendance in week 6?', 'what is the attendance for each week?', 'What was the attendance in week 12?', ' What was the attendance in week 13?']
********259***************
1 0
Thought: Based on the SQL query provided, the Yugoslavian national team lost the Balkan Cup against Romania with an aggregate score of 3:4. The aggregate score of the Yugoslavian national team against Romania in the Balkan Cup was 7, not 3:4.
Answer: 0
['what were the results of the games played by the Yugoslavian national team in the Balkan Cup?', 'When did the Yugoslavian national team lose the Balkan Cup?', ' What was the aggregate score of the Yugoslavian national team against Romania in the Balkan Cup?']
********265***************
0 1
Thought: Based on the SQL query provided, the Yugoslavian national team failed to score in world cup qualifying matches 1 time, and dropped a world cup qualify match 2:1 against Denmark.
Answer: 1
['how many times did the yugoslavian national team fail to score in world cup qualifying matches?', ' Against which team did the Yugoslavian national team drop a world cup qualify match 2:1?', 'When did the Yugoslavian national team fail to score?']
********266***************
0 1
Thought: Based on the SQL query provided, the Yugoslavian national team scored 7 goals and allowed 3 goals in the 1982 World Cup qualifiers.
Answer: 1
['How many goals did the Yugoslavian national team score in the 1982 world cup qualify?', ' How many goals did the Yugoslavian national team allow in the 1982 world cup qualify?', 'How many goals did the Yugoslavian national team score and allow in the 1982 World Cup qualifiers?']
********272***************
0 1
Thought: Based on the sub-table, March had the most games played with 7, while April had the fewest games played with 3. Therefore, the claim that March is featured more often as a month in the date than any other month, followed by the 4 games in April is true.
Answer: 1
['how many games were played in each month?', 'How many games were played in March?', ' How many games were played in April?']
********287***************
1 0
Thought: Based on the SQL query provided, there is 1 translator in the frequency MHz in the 100's and 3 translators in the frequency range of 90.0 to 99.9 MHz.
Answer: 0
["How many translators are there in the frequency mhz in the 100's?", 'how many translators are there in each frequency range?', " How many translators are there in the frequency mhz in the 90's?"]
********290***************
1 0
Thought: Based on the SQL query provided, the station w293al has an ERP W of 80, which is higher than the ERP W of w264bf, which is 2. However, the frequency_mhz of w293al is 106.5, and the frequency_mhz of w264bf is 100.7. The claim that w293al has the highest ERP W and w264bf has the lowest ERP W is false.
Answer: 0
[' What is the frequency_mhz of w264bf?', 'What is the frequency_mhz of w293al?', 'which station has the highest and lowest erp w?']
********291***************
1 0
Thought: Based on the SQL query provided, the cities of Illinois and Indiana each have 3 translators. However, the sub-table does not contain any data from the database, so we cannot verify the claim based on the provided information.
Answer: 0
[' How many translators does Indiana have?', 'How many translators does Illinois have?', 'how many translators are there in Illinois and Indiana combined?']
********292***************
0 1
Thought: Based on the sub-table, the query can be answered. There are 6 call signs that have a frequency change range of 5 MHz.
Answer: 1
['what is the frequency range for the radio stations listed?', ' How many call signs have a frequency change range of 5 MHz?', 'What are the frequency ranges for the call signs?']
********294***************
0 1
Thought: Based on the SQL query provided, the ERP_W value of 2 is the only value that appears in the table.
Answer: 1
['which call sign has an ERP of 2 W?', 'What is the frequency_mhz for the call_sign w264bf?', ' What is the city_of_license for the call_sign w264bf?']
********300***************
0 1
Thought: Based on the SQL query provided, the tournament that had the same result in the year 1990 and 1999 was the grand slam tournaments.
Answer: 1

'3 7 11 19 21 22 24 30 32 37 38 41 42 43 45 58 59 63 64 73 74 76 79 80 83 84 85 89 93 99'

In [184]:
table_name = []
for i in range(len(table_loader.dataset)):
    if table_loader.dataset[i]['table']['id'] not in table_name:
        table_name.append(table_loader.dataset[i]['table']['id'])
    else: 
        continue

In [182]:
from executor import SQLManager
from sqlalchemy import create_engine
from data_loader import TableLoader, TableFormat
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine)
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
# table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
i = 234

def show_table(data, execute=False):
    formatter = TableFormat(format='none', data=data)
    print(data['table']['id'])
    print(data['statement'])
    print(formatter.format_psql())
    # print(preds[i])
    # print(SQLs[i])
    # test_df = manager.execute_from_df(SQLs[i], formatter.all_data)
    # print(test_df)
    print(data['label'])
print(table_loader.dataset[i]['table']['caption'])
show_table(table_loader.dataset[i]) 

solheim cup
2-1546813-1.html.csv
patty sheehan , judy rankin and kathy whitworth be all captain that win the solheim cup 1 time
+--------+-----------------------------------------------+----------------+-----------+-----------------+-------------------+
|   year | venue                                         | winning_team   | score     | usa_captain     | europe_captain    |
|--------+-----------------------------------------------+----------------+-----------+-----------------+-------------------|
|   2013 | colorado golf club , colorado , usa           | europe         | 18 - 10   | meg mallon      | liselotte neumann |
|   2011 | killeen castle golf resort , ireland          | europe         | 15 - 13   | rosie jones     | alison nicholas   |
|   2009 | rich harvest farms , illinois , usa           | united states  | 16 - 12   | beth daniel     | alison nicholas   |
|   2007 | halmstad gk , sweden                          | united states  | 16 - 12   | betsy king      | helen alfr

In [11]:
for i in range(len(table_loader.dataset)):
    print(i, table_loader.dataset[i]['statement'])

0 the scheduled date for the farm with 17 turbine be 2012
1 all 12 club play a total of 22 game for the wru division one east
2 touchdown atlantic , in the category of sporting , be establish in 2010
3 the team with the most goal for in the 1986 - 87 north west county football league be flixton
4 in 1980 , 1981 and 1985 pat bradley (golfer) not have the average of 278 win score
5 polona hercog partner with alberta brianti after she have stephanie vogt as the partner
6 a gamecube game loss the award in each of the first 3 year
7 scotland be the country with the least point
8 only 1 of the college list be public , and it be in new orleans
9 19.30 million viewer watch the season finale on march 12 , 1979
10 the player garrett sutherland who be in lb position have a pick of 22
11 st joseph 's catholic school be a state integrated school in te kuiti for year 1 - 8
12 when north melbourne be the home team , they face away team footscray who score 8.9 (57)
13 gene donaldson from purdue be pic

#### show wikitable

In [ ]:
nu-3901 Al Unser, Jr. [S['rick mears']]
3898 nu-3905 Ludwig Wolf from Germany (GER) [S['ludwig wolf']]
3899 nu-3906 2 [N(10.000000)['10']]
3903 nu-3910 3 [N(7.000000)['7']]
3906 nu-3913 Survey USA [D(2007,12,17)['dec. 17, 2007']]
3915 nu-3922 2010 [N(2012.000000)['2012']]
3920 nu-3927 N/A [S['l 29-21']]
3922 nu-3929 Apollo [S['achilles']]
3923 nu-3930 Die heilige Linde [S['walamund']] 下一行存在空行，格式不规范
3924 nu-3931 Mr George Hartigan, R West [S['mr brough scott'], S['pat taaffe'], S['beltran osorio'], S['tommy carberry'], S['stan mellor']]
3929 nu-3936 0 [N(14.000000)['14 years']]
3932 nu-3939 Total number of pasurams [S['thiruvay mozhi']]
3934 nu-3941 11 [N(16.000000)['16']]               给的三行有问题，导致sql有问题
3937 nu-3944 No data available [S['16:14.1']]
3938 nu-3945 Cincinnati, Hamilton [S['cincinnati']]
3942 nu-3949 7DY [S['ulverstone']]
3944 nu-3951 September 29 at #20 Nebraska [D(-1,9,29)['september 29']]
3946 nu-3953 Giuseppe Aquaro [S['rumen trifonov']]
3948 nu-3955 Goodbye City...Hello Country [N(1.000000)['1']]
3952 nu-3959 3 [N(5.000000)['5']]
3956 nu-3963 Above [S['below']]
3962 nu-3969 128 [S['decimal128']]
3968 nu-3975 2 [N(5.000000)['5']]
3970 nu-3977 -18 [N(18.000000)['18']]
3977 nu-3984 38 [N(17.000000)['17']]
3978 nu-3985 Bob Anderson, Chris Amon [S['dan gurney']]
3982 nu-3989 No data from database [S['essentials']]
3984 nu-3991 6 [N(5.000000)['5']]
3985 nu-3992 John Lenthall [S['laramie']]
3990 nu-3997 71,886.6 [N(43674.000000)['43674']]

In [2]:
import sys
sys.path.append('/media/disk1/chatgpt/zh/tabular_data')

In [176]:
from executor import SQLManager
from sqlalchemy import create_engine
from data_loader import TableLoader, TableFormat
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine)
table_loader = TableLoader(table_name='wikitable', split='test', use_sample=False, small_test=False)
# table_loader = TableLoader(table_name='wikitable', split='test', use_sample=False, small_test=False)
i = 1441
def show_table(data, execute=False):
    formatter = TableFormat(format='none', data=data)
    print(data['id'])
    print(data['query'])
    print(formatter.format_psql())
    # print(preds[i])
    # print(SQLs[i])
    # test_df = manager.execute_from_df(SQLs[i], formatter.all_data)
    # print(test_df)
    print(data['label'])

show_table(table_loader.normalize_table(table_loader.dataset[i]) )

nu-1441
who was the first mayor to be inaugurated in witswaterand?
+---------+-------------------+----------+--------------------+----------+---------------------+
| Year    | Name              | Year_1   | Name_1             | Year_2   | Name_2              |
|---------+-------------------+----------+--------------------+----------+---------------------|
| 1903-04 | Mr B. Owen- Jones | 1935-36  | Mr W.Pearce        | 1967-68  | Mr J.F.Serfontein   |
| 1904-05 | Mr B. Owen- Jones | 1936-37  | Mr W.Pearce        | 1968-69  | Mr Ben Steyn        |
| 1905-06 | Mr G. Constable   | 1937 -38 |                    | 1969-70  |                     |
| 1907-08 | Mr T.R.Ziervogel  | 1939 -40 | Mr W.E.Vickers     | 1971-72  | Mr Chris Smith      |
| 1908-09 | Mr T.R.Ziervogel  | 1940-41  | Mr P.Venter        | 1972-73  | Mr Ben Steyn        |
| 1909-10 | Mr J.Morris       | 1941-42  | Mr P.Venter        | 1973-74  | Mr Issy Kramer      |
| 1910-11 |                   | 1942-43  | Mr P.Venter      

In [179]:
table_name = []
for i in range(len(table_loader.dataset)):
    if table_loader.dataset[i]['table']['name'] not in table_name:
        table_name.append(table_loader.dataset[i]['table']['name'])
    else:
        continue

In [38]:
for ind, i in enumerate(table_loader.dataset['question']):
    print(ind, i)

0 how many total orbit pairs are there?
1 the two level 6 seasons were 2012/13 and what other season?
2 which year had an average population of less than 38?
3 how many awards were handed out in 2004?
4 what was the most consecutive losses for the mercury in this season?
5 which year had less votes,1994 or 2009?
6 when was the first season that the team was in tier 6?
7 how many songs were juried about eight?
8 how many games were played in the month of november
9 how many encodings have a decimal above 200 for the capital letter em?
10 what municipality is on average 25 km away?
11 to which team did charleston southern give up at most 40 points?
12 what was the first year that the romanian population was less than 51,000?
13 the films beladingala baale and sparsha have which award in common?
14 how many stations opened before 1926?
15 what was the date of the only match to occur in sapporo, japan?
16 how many parties had more than 50 seats in the third duma?
17 which country won the g

In [6]:
from executor import SQLManager
from sqlalchemy import create_engine
from data_loader import TableLoader, TableFormat
table_loader = TableLoader(table_name='sqa', split='test', use_sample=True)
i = 1014
def show_table(data):
    formatter = TableFormat(format='none', data=data)
    print(data['id'])
    print(data['query'])
    print(formatter.format_psql())
    print(data['label'])
    # sum, ope = split_answer(data[ind]['predict'])
    # print(ope)
    # print(manager.format_sql(ope, table_name='DF'))
show_table(table_loader.normalize_table(table_loader.dataset[i])) 
print(all_sql[i])
print(all_data[i])

IndexError: Invalid key: 1014 is out of bounds for size 300

In [5]:
# show table query
#debugger
from data_loader import TableFormat, TableLoader
from executor import SQLManager
from sqlalchemy import create_engine
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine)
table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True)
def show_table(data):
    formatter = TableFormat(format='none', data=data)
    print(data['statement'])
    print(data['table']['caption'])
    # print(formatter.format_html())
    print(formatter.format_psql())
    print(data['label'])
    # data.columns = [manager.normalize_col_name(c) for c in formatter.all_data.columns]
    # data.to_sql('DF', manager.engine, if_exists='replace', index=False)
    
    # subtable = pd.read_sql(command, self.engine)
    # test_df = manager.execute_from_df(SQL, formatter.all_data)
    # return test_df
# test_df = show_table(table_loader.dataset[i])
show_table(table_loader.dataset[130])
for ind, i in enumerate(table_loader.dataset['statement']):
    print(ind, i)
 

ModuleNotFoundError: No module named 'data_loader'

### Evaluate Tabfact


In [160]:
import re
def extract_yes_no_and_map(text):
    # Convert the input text to lowercase for case-insensitive matching
    text = text.lower()

    # Define regular expressions for yes/no matching
    yes_patterns = [r'\byes\b', r'\btrue\b']
    no_patterns = [r'\bno\b', r'\bfalse\b']

    # Check for "0"
    if text == "0":
        return "0"

    # Check for "1"
    if text == "1":
        return "1"

    # Check for yes
    for pattern in yes_patterns:
        if re.search(pattern, text):
            return "1"

    # Check for no
    for pattern in no_patterns:
        if re.search(pattern, text):
            return "0"

    # Return 2 if neither yes nor no is found
    return "2"

def eval_fv_match(pred_list, gold_list):
        acc = 0.0
        for pred, gold in zip(pred_list, gold_list):
            pred, gold = extract_yes_no_and_map(pred), extract_yes_no_and_map(gold)
            if pred == gold:
                acc += 1
        acc = acc / len(pred_list)
        return acc
def eval_blury_string(pred_list):
    pred_label = []
    for pred in pred_list:
        predict_ans = pred.split('\n')[-1]
        if '0' in predict_ans:
            predict_ans = '0'
        elif '1' in predict_ans:
            predict_ans = '1'
        else:
            predict_ans = '2'
        pred_label.append(predict_ans)
    return pred_label

In [2]:
labels = []
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
for i in range(len(table_loader.dataset)):
    labels.append(table_loader.normalize_table(table_loader.dataset[i])['label'])

NameError: name 'TableLoader' is not defined

In [138]:
data4 = pd.read_csv('./result/answer/tabfact_05-15_10-56-31.csv')

In [139]:
data3 = pd.concat([data3, data4]).reset_index(drop=True)

In [150]:
data3.head()

,preds,statements,ids,tokens,extra,final_answer
0,"To verify whether the claim is true or false, ...",the sweet dream episode happen later in the se...,0,8860,When did the sweet dream episode happen in the...,"To verify whether the claim is true or false, ..."
1,The SQL query is filtering the table DF for ro...,david moore direct 3 episode of series 2 of me...,1,8064,How many episodes of series 2 of Merlin did Da...,The SQL query is filtering the table DF for ro...
2,The SQL query is filtering the data from the t...,the lady of the lake episode have the most uk ...,2,7948,Which episode has the most UK viewers?;What i...,The SQL query is filtering the data from the t...
3,"To verify the claim, we need to check the ""DF""...",lucy watkins only write 1 episode of series 2,3,7887,How many episodes did lucy watkins write for s...,"To verify the claim, we need to check the ""DF""..."
4,The SQL query is filtering the dataframe DF to...,beauty and the beast (part 2) have more uk vie...,4,8096,how many UK viewers did each part of Beauty an...,The SQL query is filtering the dataframe DF to...


In [161]:
import pandas as pd

data = pd.read_csv('../result/answer/tabfact_zh_04-25_15-42-17.csv')
data2 = pd.read_csv('../result/answer/tabfact_05-13_08-47-41.csv')
data3 = pd.read_csv('./result/answer/tabfact_05-16_02-20-54.csv')
data['final_answer'] = data['preds'].apply(lambda x: x.split('Answer:')[-1].strip())
data2['final_answer'] = data2['preds'].apply(lambda x: x.split('Answer:')[-1].strip())
data3['final_answer'] = data3['preds'].apply(lambda x: x.split('Answer:')[-1].strip())

In [162]:
labels = []
preds = []
ids = []
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
for i, row in data3.iterrows():
    ids.append(row['ids'])
    labels.append(table_loader.normalize_table(table_loader.dataset[int(row['ids'])])['label'])
    preds.append(row['final_answer'])

In [147]:
preds[2]

'The SQL query is filtering the data from the table DF to only include the episode titled "the lady of the lake" and then finding the maximum number of UK viewers for that episode.\n\nBased on the sub-table generated by the SQL, it shows that the episode "the lady of the lake" indeed has the maximum UK viewers of 6.3 million.\n\nTherefore, the provided claim/query is true.\n\nReturn: 1'

In [163]:
eval_fv_match(preds, labels)

0.8367139959432048

In [51]:
for i, row in data.iterrows():
    
    label = extract_yes_no_and_map(table_loader.normalize_table(table_loader.dataset[int(row['ids'])])['label'])
    pred3 = extract_yes_no_and_map(row['final_answer'])
    if label != pred3:
        id = row['ids']
        print(f'********{id}***************')
        
        print(label, pred3)
        print(row['preds'])
        print(row['extra'].split(';'))
    

********1***************
1 0
Thought: Based on the sub-table, it appears that David Moore did not direct any episodes of series 2 of Merlin. Therefore, the answer to the query is 0 episodes.
Answer: 0
[' Which episodes did David Moore direct for series 2 of Merlin?', 'How many episodes did David Moore direct for series 2 of Merlin?', 'how many episodes of series 2 of Merlin did David Moore direct?']
********3***************
1 0
Thought: Based on the SQL query provided, Lucy Watkins wrote 0 episodes for series 2. 
Answer: 0
[' Which episode did Lucy Watkins write for series 2?', 'how many episodes of series 2 did lucy watkins write?', 'How many episodes did Lucy Watkins write for series 2?']
********8***************
0 1
Thought: Based on the SQL query provided, there is no data from the database that matches the condition of Lucy Watkins writing 6 episodes of series 2. 
Answer: 1
['how many episodes did lucy watkins write for series 2?', 'How many episodes did Lucy Watkins write for ser

In [11]:

eval_fv_match(preds, labels)


0.8241256969082615

In [64]:
acc

227

In [93]:
all_data.head()

,preds_1,statements_1,final_answer_1,preds_2,statements_2,final_answer_2,preds,statements,tokens,extra,final_answer
ids,,,,,,,,,,,
0,"Thought: Based on the SQL query provided, the ...",the sweet dream episode happen later in the se...,1,"Thought: The ""Sweet Dreams"" episode happened o...",the sweet dream episode happen later in the se...,1,"Thought: Based on the SQL query provided, the ...",the sweet dream episode happen later in the se...,7622,"What was the air date of the episode ""The Witc...",1
1,"Thought: Based on the SQL query provided, ther...",david moore direct 3 episode of series 2 of me...,0,"Thought: Based on the SQL query provided, ther...",david moore direct 3 episode of series 2 of me...,0,"Thought: Based on the SQL query provided, ther...",david moore direct 3 episode of series 2 of me...,5548,How many episodes did David Moore direct in se...,0
2,"Thought: Based on the SQL query provided, the ...",the lady of the lake episode have the most uk ...,1,"Thought: Based on the SQL query provided, the ...",the lady of the lake episode have the most uk ...,1,"Thought: Based on the SQL query provided, the ...",the lady of the lake episode have the most uk ...,5608,What was the title of the episode with the hig...,1
3,"Thought: Based on the SQL query provided, ther...",lucy watkins only write 1 episode of series 2,0,"Thought: Based on the SQL query provided, Lucy...",lucy watkins only write 1 episode of series 2,0,"Thought: Based on the SQL query provided, ther...",lucy watkins only write 1 episode of series 2,5484,How many episodes did Lucy Watkins write for s...,0
4,"Thought: Based on the SQL query provided, the ...",beauty and the beast (part 2) have more uk vie...,1,"Thought: Based on the SQL query provided, the ...",beauty and the beast (part 2) have more uk vie...,1,"Thought: Based on the SQL query provided, the ...",beauty and the beast (part 2) have more uk vie...,7682,"What was the original air date of ""the curse o...",1


In [90]:
all_data = data.set_index(keys='ids').join(data2.set_index(keys='ids'), how='inner',lsuffix='_1', rsuffix='_2').join(data3.set_index(keys='ids'), how='inner',lsuffix='_1', rsuffix='_3')

In [100]:
acc

144

In [102]:
acc

1628

In [104]:
from collections import Counter
labels = []
preds = []
ids = []
acc = 0
err = 0
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
for i, row in all_data.iterrows():
    label = extract_yes_no_and_map(table_loader.normalize_table(table_loader.dataset[int(i)])['label'])
    pred1 = extract_yes_no_and_map(row['final_answer_1'])
    pred2 = extract_yes_no_and_map(row['final_answer_2'])
    pred3 = extract_yes_no_and_map(row['final_answer'])
    print(Counter([pred1,pred2,pred3]).most_common())
    if Counter([pred1,pred2,pred3]).most_common()[0][0] == label:
        acc += 1
    else:
        print
    if pred1 != label and pred2 != label and pred3 != label:
        # print(label, pred1, pred2)
        err += 1


[('1', 3)]
[('0', 3)]
[('1', 3)]
[('0', 3)]
[('1', 3)]
[('0', 3)]
[('0', 3)]
[('0', 2), ('1', 1)]
[('0', 3)]
[('0', 3)]
[('0', 3)]
[('0', 3)]
[('0', 3)]
[('0', 2), ('1', 1)]
[('0', 2), ('1', 1)]
[('0', 3)]
[('1', 2), ('0', 1)]
[('0', 2), ('1', 1)]
[('0', 2), ('1', 1)]
[('0', 3)]
[('1', 3)]
[('1', 2), ('0', 1)]
[('0', 3)]
[('1', 3)]
[('1', 2), ('0', 1)]
[('0', 3)]
[('1', 2), ('0', 1)]
[('0', 3)]
[('0', 3)]
[('0', 3)]
[('1', 3)]
[('1', 3)]
[('1', 2), ('0', 1)]
[('1', 3)]
[('1', 3)]
[('0', 3)]
[('0', 3)]
[('0', 3)]
[('0', 2), ('1', 1)]
[('1', 2), ('0', 1)]
[('1', 3)]
[('0', 2), ('1', 1)]
[('0', 2), ('2', 1)]
[('1', 2), ('0', 1)]
[('1', 3)]
[('0', 2), ('1', 1)]
[('1', 2), ('0', 1)]
[('1', 2), ('0', 1)]
[('0', 3)]
[('1', 3)]
[('1', 3)]
[('0', 2), ('1', 1)]
[('1', 3)]
[('1', 3)]
[('0', 3)]
[('0', 2), ('1', 1)]
[('0', 3)]
[('0', 3)]
[('0', 2), ('1', 1)]
[('0', 3)]
[('1', 3)]
[('1', 3)]
[('0', 3)]
[('1', 3)]
[('1', 3)]
[('0', 3)]
[('1', 2), ('0', 1)]
[('0', 3)]
[('1', 3)]
[('0', 2), ('1', 1)]


In [134]:
acc = 0
errors = []
for i in range(len(preds)):
    pred, gold = extract_yes_no_and_map(preds[i]), extract_yes_no_and_map(labels[i])
    if pred != gold:
        print(ids[i], pred, gold)
        errors.append(i)
    else:
        acc += 1

1 0 1
3 0 1
7 1 0
11 0 1
14 0 1
16 1 0
17 1 0
21 0 1
22 0 1
26 0 1
32 0 1
37 0 1
39 1 0
42 0 1
43 1 0
46 1 0
47 1 0
51 0 1
58 1 0
59 0 1
62 0 1
63 1 0
64 1 0
66 1 0
69 0 1
72 0 1
73 1 0
76 1 0
79 0 1
80 0 1
82 0 1
84 1 0
87 1 0
91 0 1
100 0 1
101 0 1
103 1 0
106 1 0
107 1 0
108 0 1
115 2 0
121 1 0
126 1 0
133 1 0
135 1 0
138 1 0
140 0 1
148 0 1
151 0 1
160 0 1
169 0 1
180 0 1
181 1 0
184 1 0
188 0 1
192 1 0
196 1 0
199 0 1
210 0 1
211 0 1
213 1 0
216 2 0
225 1 0
227 1 0
236 1 0
240 0 1
241 0 1
249 0 1
254 1 0
259 0 1
261 0 1
265 1 0
268 0 1
276 1 0
277 1 0
281 0 1
291 0 1
292 1 0
296 1 0
300 1 0
301 0 1
309 0 1
315 0 1
341 1 0
349 1 0
350 1 0
355 0 1
362 1 0
375 0 1
390 1 0
392 1 0
393 1 0
396 1 0
397 1 0
398 1 0
408 0 1
422 1 0
425 0 1
426 1 0
430 1 0
431 1 0
432 1 0
435 0 1
436 0 1
438 0 1
439 0 1
441 1 0
442 1 0
444 1 0
457 1 0
463 1 0
473 1 0
475 0 1
476 0 1
488 0 1
489 0 1
491 1 0
493 1 0
495 0 1
499 0 1
502 1 0
503 1 0
504 0 1
510 1 0
511 1 0
517 0 1
520 0 1
522 1 0
534 1 0
538 0

In [ ]:
data = pd.read_csv('./result/answer/tabfact_05-10_07-12-50.csv')
data['final_answer'] = data['preds'].apply(lambda x: x.split('Answer:')[-1].strip())

if len(predict_ans) == 0:
    predict_ans = json.loads(l)['pred'][-3:]
    if '0' in predict_ans:
        predict_ans = '0'
    elif '1' in predict_ans:
        predict_ans = '1'
    else:
        predict_ans = '2'
    pred_label.append(predict_ans)
if verbose:
    print(pred_label)
    print(gold_list)
    print(np.where(np.array(pred_label) != np.array(gold_list)))
return eval_fv_match(pred_label, gold_list)

10
10


### Evaluate WikiTable

In [27]:
import re
import string
import recognizers_suite
import recognizers_suite as Recognizers
from recognizers_text import Culture, ModelResult
def str_normalize(user_input, recognition_types=None):
    """A string normalizer which recognize and normalize value based on recognizers_suite"""
    user_input = str(user_input)
    user_input = user_input.replace("\\n", "; ")

    def replace_by_idx_pairs(orig_str, strs_to_replace, idx_pairs):
        assert len(strs_to_replace) == len(idx_pairs)
        last_end = 0
        to_concat = []
        for idx_pair, str_to_replace in zip(idx_pairs, strs_to_replace):
            to_concat.append(orig_str[last_end : idx_pair[0]])
            to_concat.append(str_to_replace)
            last_end = idx_pair[1]
        to_concat.append(orig_str[last_end:])
        return ''.join(to_concat)

    if recognition_types is None:
        recognition_types = [
            "datetime",
            "number",
            "ordinal",
            "percentage",
            "age",
            "currency",
            "dimension",
            "temperature",
        ]
    culture = Culture.English
    for recognition_type in recognition_types:
        if re.match("\d+/\d+", user_input):
            # avoid calculating str as 1991/92
            continue
        recognized_list = getattr(
            recognizers_suite, "recognize_{}".format(recognition_type)
        )(
            user_input, culture
        )  # may match multiple parts
        strs_to_replace = []
        idx_pairs = []
        for recognized in recognized_list:
            if not recognition_type == 'datetime':
                recognized_value = recognized.resolution['value']
                if str(recognized_value).startswith("P"):
                    # if the datetime is a period:
                    continue
                else:
                    strs_to_replace.append(recognized_value)
                    idx_pairs.append((recognized.start, recognized.end + 1))
            else:
                if recognized.resolution:  # in some cases, this variable could be none.
                    if len(recognized.resolution['values']) == 1:
                        strs_to_replace.append(
                            recognized.resolution['values'][0]['timex']
                        )  # We use timex as normalization
                        idx_pairs.append((recognized.start, recognized.end + 1))

        if len(strs_to_replace) > 0:
            user_input = replace_by_idx_pairs(user_input, strs_to_replace, idx_pairs)

    if re.match("(.*)-(.*)-(.*) 00:00:00", user_input):
        user_input = user_input[: -len("00:00:00") - 1]
        # '2008-04-13 00:00:00' -> '2008-04-13'
    return user_input


def normalize_answer(s):
    def remove_articles(text):
        return re.sub(re.compile(r"\b(a|an|the)\b", re.UNICODE), " ", text)

    def whilt_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return whilt_space_fix(remove_articles(remove_punc(lower(s))))


In [28]:

import sys, os, re, argparse
import unicodedata
from codecs import open
from math import isnan, isinf
from abc import ABCMeta, abstractmethod


################ String Normalization ################

def normalize(x):
    if not isinstance(x, str):
        x = x.decode('utf8', errors='ignore')
    # Remove diacritics
    x = ''.join(c for c in unicodedata.normalize('NFKD', x)
                if unicodedata.category(c) != 'Mn')
    # Normalize quotes and dashes
    x = re.sub(r"[‘’´`]", "'", x)
    x = re.sub(r"[“”]", "\"", x)
    x = re.sub(r"[‐‑‒–—−]", "-", x)
    while True:
        old_x = x
        # Remove citations
        x = re.sub(r"((?<!^)\[[^\]]*\]|\[\d+\]|[•♦†‡*#+])*$", "", x.strip())
        # Remove details in parenthesis
        x = re.sub(r"(?<!^)( \([^)]*\))*$", "", x.strip())
        # Remove outermost quotation mark
        x = re.sub(r'^"([^"]*)"$', r'\1', x.strip())
        if x == old_x:
            break
    # Remove final '.'
    if x and x[-1] == '.':
        x = x[:-1]
    # Collapse whitespaces and convert to lower case
    x = re.sub(r'\s+', ' ', x, flags=re.U).lower().strip()
    return x


################ Value Types ################

class Value(object):
    __metaclass__ = ABCMeta

    # Should be populated with the normalized string
    _normalized = None

    @abstractmethod
    def match(self, other):
        """Return True if the value matches the other value.
        Args:
            other (Value)
        Returns:
            a boolean
        """
        pass

    @property
    def normalized(self):
        return self._normalized


class StringValue(Value):

    def __init__(self, content):
        assert isinstance(content, str)
        self._normalized = normalize(content)
        self._hash = hash(self._normalized)

    def __eq__(self, other):
        return isinstance(other, StringValue) and self.normalized == other.normalized

    def __hash__(self):
        return self._hash

    def __str__(self):
        return 'S' + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        return self.normalized == other.normalized


class NumberValue(Value):

    def __init__(self, amount, original_string=None):
        assert isinstance(amount, (int, float))
        if abs(amount - round(amount)) < 1e-6:
            self._amount = int(amount)
        else:
            self._amount = float(amount)
        if not original_string:
            self._normalized = str(self._amount)
        else:
            self._normalized = normalize(original_string)
        self._hash = hash(self._amount)

    @property
    def amount(self):
        return self._amount

    def __eq__(self, other):
        return isinstance(other, NumberValue) and self.amount == other.amount

    def __hash__(self):
        return self._hash

    def __str__(self):
        return ('N(%f)' % self.amount) + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, NumberValue):
            return abs(self.amount - other.amount) < 1e-6
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a number.
        Return:
            the number (int or float) if successful; otherwise None.
        """
        try:
            return int(text)
        except:
            try:
                amount = float(text)
                assert not isnan(amount) and not isinf(amount)
                return amount
            except:
                return None


class DateValue(Value):

    def __init__(self, year, month, day, original_string=None):
        """Create a new DateValue. Placeholders are marked as -1."""
        assert isinstance(year, int)
        assert isinstance(month, int) and (month == -1 or 1 <= month <= 12)
        assert isinstance(day, int) and (day == -1 or 1 <= day <= 31)
        assert not (year == month == day == -1)
        self._year = year
        self._month = month
        self._day = day
        if not original_string:
            self._normalized = '{}-{}-{}'.format(
                year if year != -1 else 'xx',
                month if month != -1 else 'xx',
                day if day != '-1' else 'xx')
        else:
            self._normalized = normalize(original_string)
        self._hash = hash((self._year, self._month, self._day))

    @property
    def ymd(self):
        return (self._year, self._month, self._day)

    def __eq__(self, other):
        return isinstance(other, DateValue) and self.ymd == other.ymd

    def __hash__(self):
        return self._hash

    def __str__(self):
        return (('D(%d,%d,%d)' % (self._year, self._month, self._day))
                + str([self._normalized]))

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, DateValue):
            return self.ymd == other.ymd
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a date.
        Return:
            tuple (year, month, date) if successful; otherwise None.
        """
        try:
            ymd = text.lower().split('-')
            assert len(ymd) == 3
            year = -1 if ymd[0] in ('xx', 'xxxx') else int(ymd[0])
            month = -1 if ymd[1] == 'xx' else int(ymd[1])
            day = -1 if ymd[2] == 'xx' else int(ymd[2])
            assert not (year == month == day == -1)
            assert month == -1 or 1 <= month <= 12
            assert day == -1 or 1 <= day <= 31
            return (year, month, day)
        except:
            return None


################ Value Instantiation ################

def to_value(original_string, corenlp_value=None):
    """Convert the string to Value object.
    Args:
        original_string (basestring): Original string
        corenlp_value (basestring): Optional value returned from CoreNLP
    Returns:
        Value
    """
    if isinstance(original_string, Value):
        # Already a Value
        return original_string
    if not corenlp_value:
        corenlp_value = original_string
    # Number?
    amount = NumberValue.parse(corenlp_value)
    if amount is not None:
        return NumberValue(amount, original_string)
    # Date?
    ymd = DateValue.parse(corenlp_value)
    if ymd is not None:
        if ymd[1] == ymd[2] == -1:
            return NumberValue(ymd[0], original_string)
        else:
            return DateValue(ymd[0], ymd[1], ymd[2], original_string)
    # String.
    return StringValue(original_string)


def to_value_list(original_strings, corenlp_values=None):
    """Convert a list of strings to a list of Values
    Args:
        original_strings (list[basestring])
        corenlp_values (list[basestring or None])
    Returns:
        list[Value]
    """
    assert isinstance(original_strings, (list, tuple, set))
    if corenlp_values is not None:
        assert isinstance(corenlp_values, (list, tuple, set))
        assert len(original_strings) == len(corenlp_values)
        return list(set(to_value(x, y) for (x, y)
                        in zip(original_strings, corenlp_values)))
    else:
        return list(set(to_value(x) for x in original_strings))


################ Check the Predicted Denotations ################

def check_denotation(target_values, predicted_values):
    """Return True if the predicted denotation is correct.

    Args:
        target_values (list[Value])
        predicted_values (list[Value])
    Returns:
        bool
    """
    # Check size
    if len(target_values) != len(predicted_values):
        return False
    # Check items
    for target in target_values:
        if not any(target.match(pred) for pred in predicted_values):
            return False
    return True
def tsv_unescape(x):
    """Unescape strings in the TSV file.
    Escaped characters include:
        newline (0x10) -> backslash + n
        vertical bar (0x7C) -> backslash + p
        backslash (0x5C) -> backslash + backslash

    Args:
        x (str or unicode)
    Returns:
        a unicode
    """
    return x.replace(r'\n', '\n').replace(r'\p', '|').replace('\\\\', '\\')

def tsv_unescape_list(x):
    """Unescape a list in the TSV file.
    List items are joined with vertical bars (0x5C)

    Args:
        x (str or unicode)
    Returns:
        a list of unicodes
    """
    return [tsv_unescape(y) for y in x.split('|')]

In [29]:
import os
tagged_dataset_path = '../result/data/wikitable/tagged/data'
target_values_map = {}
for filename in os.listdir(tagged_dataset_path):
    if filename[0]=='.':
        continue
    filename = os.path.join(tagged_dataset_path, filename)
    print('Reading dataset from', filename)
    with open(filename, 'r') as fin:
        header = fin.readline().rstrip('\n').split('\t')
        for line in fin:
            stuff = dict(zip(header, line.rstrip('\n').split('\t')))
            ex_id = stuff['id']
            original_strings = tsv_unescape_list(stuff['targetValue'])
            canon_strings = tsv_unescape_list(stuff['targetCanon'])

            target_values_map[ex_id] = to_value_list(
                original_strings, canon_strings)

Reading dataset from ../result/data/wikitable/tagged/data/training.tagged
Reading dataset from ../result/data/wikitable/tagged/data/pristine-unseen-tables.tagged
Reading dataset from ../result/data/wikitable/tagged/data/pristine-seen-tables.tagged


In [ ]:
st2id = {}
with open(os.path.join('../../datasets/wtq','valid_lower.jsonl')) as f:
    lines = f.readlines()
    for l in lines:
        dic = json.loads(l)
        st = dic['statement']
        ids = dic['ids']
        st2id[st] = ids

##### read_csv

In [198]:
import pandas as pd
# data1 = pd.read_csv('./result/answer/wikitable_zh_05-03_15-27-43.csv', encoding='utf-8') 
# data2 = pd.read_csv('./result/answer/wikitable_zh_05-03_15-46-07.csv', encoding='utf-8') 
data = pd.read_csv('../result/answer/wikitable_05-12_14-25-15.csv')
data2 = pd.read_csv('../result/answer/wikitable_05-14_08-17-04.csv')
data3 = pd.read_csv('../result/answer/wikitable_05-15_12-49-12.csv')

In [97]:
data = pd.concat([data1, data2], axis=0).reset_index()

In [199]:
data['final_answer'] = data['preds'].apply(lambda x: x.split('Answer:')[-1].strip())
data2['final_answer'] = data2['preds'].apply(lambda x: x.split('Answer:')[-1].strip())
data3['final_answer'] = data3['preds'].apply(lambda x: x.split('Answer:')[-1].strip())

In [173]:
all_data = data2.set_index('ids').join(data3.set_index('ids'), how='inner', lsuffix='_1', rsuffix='_2').join(data.set_index('ids'), how='inner')

In [174]:
all_data.head()

,preds_1,statements_1,tokens_1,extra_1,final_answer_1,preds_2,statements_2,tokens_2,extra_2,final_answer_2,preds,statements,sample_n,final_answer
ids,,,,,,,,,,,,,,
nu-0,Thought: The SQL query filtered the top 10 cyc...,which country had the most cyclists finish wit...,5077,Which country were the cyclists who finished w...,ITA,"Thought: Based on the SQL query provided, Ital...",which country had the most cyclists finish wit...,5077,which country had the most cyclists finish wi...,ITA,Thought: The SQL query first extracts the last...,which country had the most cyclists finish wit...,NaN,ITA
nu-1,"Thought: Based on the SQL query provided, 100,...",how many people were murdered in 1940/41?,6199,How many people were murdered in 1940/41?;how ...,"100,000",Thought: The SQL query filtered the data to on...,how many people were murdered in 1940/41?,5864,how many people were murdered in 1940?;How man...,100000,"Thought: Based on the SQL query provided, the ...",how many people were murdered in 1940/41?,NaN,100000
nu-2,Thought: The SQL query filters the data to onl...,how long did it take for the new york american...,5223,When did the New York Americans win the Nation...,1953/54,Thought: The New York Americans won the Nation...,how long did it take for the new york american...,4971,When did the New York Americans win the Nation...,17 years,Thought: The SQL query filters the table to on...,how long did it take for the new york american...,NaN,17 years
nu-3,"Thought: The episode ""Donnell's Birthday Party...",alfie's birthday party aired on january 19. wh...,8830,What was the airdate of the episode after Alfi...,"January 20, 1995",Thought: The SQL query is filtering for the or...,alfie's birthday party aired on january 19. wh...,6850,what was the airdate of the episode that aire...,No data from database,Thought: The original air date of the episode ...,alfie's birthday party aired on january 19. wh...,NaN,1995-01-26
nu-4,Thought: The SQL query is filtering the DataFr...,what is the number of 1st place finishes acros...,5209,How many first place finishes were achieved in...,17,Thought: The SQL query is counting the number ...,what is the number of 1st place finishes acros...,5218,How many first place finishes were achieved in...,17,Thought: The total number of 1st place finishe...,what is the number of 1st place finishes acros...,NaN,17


In [175]:
acc = 0
from collections import Counter
for index, row in all_data.iterrows():
    target = target_values_map[index]
    answer = Counter([row['final_answer_1'], row['final_answer_2'], row['final_answer']]).most_common()[0][0]
    
    if len(target) > 1:
        pred_answer_1 = to_value_list(answer.split(','))
        # pred_answer_2 = to_value_list(row['final_answer_2'].split(','))
        # pred_answer_3 = to_value_list(row['final_answer'].split(','))
    else:
        pred_answer_1 = to_value_list([answer])
        # pred_answer_2 = to_value_list([row['final_answer_2']])
        # pred_answer_3 = to_value_list([row['final_answer']])
    if check_denotation(pred_answer_1, target):
        acc += 1
    else:
        # print(index, row['final_answer_left'],row['final_answer_right'], target)
        pass
print(acc / len(all_data))

0.6758445164275798


In [ ]:
data

#### list


In [200]:
acc = 0

for index, row in data3.iterrows():
    id = row['ids']
    target = target_values_map[id]
    
    preds = [row['final_answer']]
    if len(target) > 1:
        pred_answer = to_value_list(row['final_answer'].split(','))
    else:
        pred_answer = to_value_list(preds)
    
    if check_denotation(pred_answer, target):
        acc += 1
    else:
        print(index,row['ids'], row['final_answer'], target)
        pass
print(acc / len(data))

0 nu-0 ITA [S['italy']]
3 nu-3 No data from database [D(1995,1,26)['january 26, 1995']]
10 nu-10 1970, 1988, 1979 [N(2004.000000)['2004'], N(2005.000000)['2005'], N(2006.000000)['2006']]
11 nu-11 Jamie Cureton [S['john']]
12 nu-12 17 [N(440.000000)['440']]
13 nu-13 0 [N(7.000000)['7']]
14 nu-14 \\0 [S['space']]
15 nu-15 305 [N(68.000000)['68']]
19 nu-19 3 [N(492111.000000)['492,111']]
23 nu-23 Sayonara [S['brindabella']]
25 nu-25 8 [N(3.000000)['3']]
26 nu-26 14 [N(20.000000)['20']]
29 nu-29 Once [N(1.000000)['1']]
34 nu-34 No other contestant(s) [S['patricia valiahmetova'], S['carmen jenockova'], S['mariesea mnesicu'], S['jahaira novgorodova'], S['anastasija larkova']]
35 nu-35 7 [N(0.000000)['0']]
36 nu-36 1 [N(4.000000)['4']]
37 nu-37 0 [N(10.000000)['10']]
42 nu-42 St Nikolai [S["st. mary's church"]]
44 nu-44 2014 [N(1992.000000)['1992']]
46 nu-46 12 [N(20.000000)['20']]
53 nu-53  [N(1935.000000)['1935']]
58 nu-58 David Berger [S['esther shahamorov']]
60 nu-60 Kranjska Gora Sloveni

In [130]:
len(data)

1974

In [30]:
acc = 0
for index, row in data3[:100].iterrows():
    id = row['ids']
    target = target_values_map[id]
    
    preds = [row['final_answer']]
    pred_answer = to_value_list(preds)
    if check_denotation(pred_answer, target):
        acc += 1
    else:
        print(index,row['ids'], row['final_answer'], target)
        pass
print(acc)

1 nu-1001 4, 5, 5 [N(12.000000)['12']]
5 nu-1005 -82 [N(1514069.000000)['1,514,069']]
6 nu-1006 less than 6 goals [S['less']]
8 nu-1008 The Bulldogs performed better in the 1906 season than the 1905 season. [S['worse']]
10 nu-1010 1992-08-15 [D(1992,8,29)['29 august 1992']]
14 nu-1014 111 [N(110.000000)['110']]
24 nu-1024 16 [N(13.000000)['13 weeks']]
31 nu-1031 2.875 [N(2.500000)['2.5']]
32 nu-1032 Jan Bos [S['yevgeny lalenkov']]
35 nu-1035 16 [N(17.000000)['17']]
36 nu-1036 Yelena Kondulaynen [S['yelena kondulaynen 44.the actress']]
37 nu-1037 0 [N(2.000000)['2']]
41 nu-1041 Sidney Smith, Ralph Foster, Sam Costen, L. S. LeTellier, George C. Rogers, or Harvey O'Brien. [S['sidney smith']]
43 nu-1043 Jack Brabham, Mike Parkes, Denny Hulme, Jochen Rindt, Dan Gurney [S['dan gurney'], S['denny hulme'], S['jochen rindt'], S['mike parkes'], S['jack brabham']]
47 nu-1047 Who Do You Think You Are [S["everything you've done wrong"]]
49 nu-1049 Matthew Nowicki [S['charles eames']]
52 nu-1052 27.

In [142]:
acc / 168

0.42857142857142855

In [31]:
culture = Culture.English
Recognizers.recognize_number('The total number of competitions is 20.', culture)[0].resolution['value']

'20'

In [ ]:
print(output.content)

The table provides information about different games played by a football team. It includes details like the week number, date of the game, opponent team, game result, game site, NFL recap, and the attendance for each game.


In [27]:
str_normalize('The total number of competitions is 20.')

'The total number of competitions is 20.'

### Experiement

#### Large table analysis

In [188]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-0125")



[83, 1609, 5963, 374, 2294, 758]

In [192]:
len(table_loader.dataset)

4344

In [194]:
table_loader = TableLoader(table_name='wikitable', split='test', use_sample=False, small_test=False)
small_tables  = []
middle_tables  = []
large_tables  = []
for i in range(len(table_loader.dataset)):
    sample = table_loader.normalize_table(
                    table_loader.dataset[i])
    formatter = TableFormat(format='none', data=sample, save_embedding=False)
    sample_data = formatter.get_sample_data(sample_type='all')
    table_length = len(encoding.encode(TableFormat.format_html(data=sample_data, table_caption=sample['table']['caption'])))
    if table_length < 2000:
        small_tables.append(sample['id'])
    elif table_length < 4000:
        middle_tables.append(sample['id'])
    else:
        large_tables.append(sample['id'])

In [195]:
len(small_tables)

3466

In [204]:
acc_small = 0
count  = 0
for index, row in data3.iterrows():
    id = row['ids']
    if id not in large_tables:
        continue
    count += 1
    target = target_values_map[id]
    
    if len(target) > 1:
        pred_answer = to_value_list(row['final_answer'].split(','))
    else:
        pred_answer = to_value_list([row['final_answer']])
    
    if check_denotation(pred_answer, target):
        acc_small += 1
    else:
        # print(index,row['ids'], row['final_answer'], target)
        pass
print(acc_small / count)

0.625
